# Scraping NM Well data

In [1]:
# The requests library is 'the farm'...it's where we'll get the raw html we'll need to parse/navigate to make our soup 
# (i.e. get our data) 
# Here's  some documentation you'll need to be familiar with
# http://docs.python-requests.org/en/master/user/quickstart/
import requests
baseUrl = 'https://wwwapps.emnrd.state.nm.us/ocd/ocdpermitting/Data/'
searchUrl = 'https://wwwapps.emnrd.state.nm.us/ocd/ocdpermitting/Data/WellSearchResults.aspx?OperatorSearchClause=BeginsWith&WellSearchClause=Contains&prod_prop_name=H&WellNumberSearchClause=BeginsWith&PoolSearchClause=Contains&Pool_name=Bone%20Spring&section=00&Status=A&CancelledAPDs=Exclude&PluggedWells=Exclude&SearchLocation=Surface'

page = requests.get(searchUrl)

contents = page.content
from bs4 import BeautifulSoup
soup = BeautifulSoup(contents, 'lxml')
exportQueryString = soup.find_all('a', class_='btn')[1]

exportFile = requests.get(baseUrl+exportQueryString.get('href'), allow_redirects=True)


In [2]:
# The code above/below will return the results of the search you specify on the ocd website. 
# now you need to take the api #'s that it returned and get the production data from the 'details' page
# That shouldn't be too difficult as you've already put that together in the cell down further. 
import csv

apiTable = BeautifulSoup(exportFile.content, "lxml")
rows = apiTable.find_all('tr')
headers = [th.text for th in apiTable.select("tr th")]

with open("searchResults.csv", "w", encoding = "utf-8") as f:
    wr = csv.writer(f)
    #wr.writerow(tableHeader)
    wr.writerow(headers)
    wr.writerows([[td.text for td in row.find_all("td")] for row in apiTable.select("tr + tr")])

In [ ]:
## Now we're going to loop through each of the api's in the searchResults.csv and get the 
## production data from the details page
import pandas as pd
detailsBaseUrl = 'https://wwwapps.emnrd.state.nm.us/ocd/ocdpermitting/Data/WellDetails.aspx?api='
searchResultData = pd.read_csv('searchResults.csv')

apiNumbers = searchResultData['API']

with open("productionDetails.csv", "w", encoding = "utf-8") as f:
    wr = csv.writer(f)
    for apiNumber in apiNumbers:
        detailsUrl = detailsBaseUrl+apiNumber

        detailsPage = requests.get(detailsUrl)
        soup = BeautifulSoup(detailsPage.content, "lxml")
        table = soup.find('table', class_ = 'tblData productionresults')
        rows = table.find_all('tr')
        headers = [th.text.strip() for th in table.select("tr th")]
        tableHead = headers[3:8]
        wr.writerow(['Production for API: ' + apiNumber])
        wr.writerow(tableHead)
        for tr in rows:
            td = tr.find_all('td')
            row = [i.text.strip() for i in td[:5]]
            wr.writerow(row)


In [41]:
# Now that we have the raw ingredients we need to make them into something
# That's where Beautiful Soup library comes in.  BS comes as a standard parser in Python
# The parsing isn't the fastest, but it's a good starting point.  If you decide you need more speed
# you can always switch out BS for something faster. 
# Here's some documentation you'll need to be familiar with:
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/
# And a short example
# https://gist.github.com/bradmontgomery/1872970
import csv
from bs4 import BeautifulSoup

contents = requests.get('https://wwwapps.emnrd.state.nm.us/ocd/ocdpermitting/Data/WellDetails.aspx?api=30-025-36992').content
soup = BeautifulSoup(contents, 'lxml')

#tableHeader = soup.find('h1', span = 'title') #[0].text.strip() #, title_ = 'API, Property Name and Well Number')

#with open("out.csv", "w", encoding = "utf-8") as f:
#    wr = csv.writer(f)
#    #wr.writerow(tableHeader)
#    wr.writerow(headers)
#    wr.writerows([[td.text for td in row.find_all("td")] for row in table.select("tr + tr")])
   


Production
['Time Frame', 'Oil(BBLS)', 'Gas(MCF)', 'Water(BBLS)', 'Days P/I']
[]
[]
['2006']
['LUSK;MORROW (GAS)']
['Mar', '729', '15676', '32', '14']
['Apr', '2856', '56356', '297', '29']
['May', '2685', '57146', '348', '31']
['Jun', '2444', '52081', '473', '30']
['Jul', '2250', '51155', '521', '31']
['Aug', '2056', '48101', '565', '31']
['Sep', '2010', '47061', '606', '30']
['Oct', '1642', '48760', '625', '31']
['Nov', '1810', '45735', '513', '29']
['Dec', '1400', '43440', '496', '30']
['Pool Total:', '19882', '465511', '4476', '286']
['2006\nAnnual Total:', '19882', '465511', '4476', '286']
['2007']
['LUSK;MORROW (GAS)']
['Jan', '1440', '42550', '488', '26']
['Feb', '1171', '38898', '460', '28']
['Mar', '1028', '35644', '364', '29']
['Apr', '956', '33849', '426', '30']
['May', '854', '31543', '393', '31']
['Jun', '496', '27227', '504', '30']
['Jul', '617', '25245', '452', '31']
['Aug', '450', '22529', '399', '31']
['Sep', '284', '19201', '335', '30']
['Oct', '249', '17311', '458', '

In [ ]:
# The 'Salad' 
# Lxml can pretty much do what BeuatifulSoup can, but it has a much faster parsing engine
# Here's the official documentation
# http://lxml.de/index.html#introduction
# Here's a user-friendly quick start guide
# http://docs.python-guide.org/en/latest/scenarios/scrape/

from lxml import html

tree = html.fromstring(page.content)

# tree now contains the whole html page in a nice tree format that we access using XPath or CSSSelect
# we'll focus on the former.  XPath is a way of locating information in structured documents such as HTML and XML
# here's a good introduction:  http://www.w3schools.com/xml/xpath_intro.asp



In [ ]:
# XPath for the first API text box
//*[@id="txtCounty"]

# XPath for the second API text box
//*[@id="txtAPI"]

# XPath for the 'Search' button
//*[@id="btnRefIDContinue"]

# XPath for the table of production data
//*[@id="data_details"]/fieldset[10]/table

# XPath for Page Header
//*[@id="ctl00_ctl00__main_main_lblApi"]

# Using Paramiko to connect to OCD's sftp

In [ ]:
import paramiko

In [ ]:
def create_sftp_client(host, port, username, password, keyfilepath, keyfiletype):
    """
    create_sftp_client(host, port, username, password, keyfilepath, keyfiletype) -> SFTPClient
 
    Creates a SFTP client connected to the supplied host on the supplied port authenticating as the user with
    supplied username and supplied password or with the private key in a file with the supplied path.
    If a private key is used for authentication, the type of the keyfile needs to be specified as DSA or RSA.
    :rtype: SFTPClient object.
    """
    sftp = None
    key = None
    transport = None
    try:
        if keyfilepath is not None:
            # Get private key used to authenticate user.
            if keyfiletype == 'DSA':
                # The private key is a DSA type key.
                key = paramiko.DSSKey.from_private_key_file(keyfilepath)
            else:
                # The private key is a RSA type key.
                key = paramiko.RSAKey.from_private_key(keyfilepath)
 
        # Create Transport object using supplied method of authentication.
        transport = paramiko.Transport((host, port))
        transport.connect(None, username, password, key)
 
        sftp = paramiko.SFTPClient.from_transport(transport)
 
        return sftp
    except Exception as e:
        print('An error occurred creating SFTP client: %s: %s' % (e.__class__, e))
        if sftp is not None:
            sftp.close()
        if transport is not None:
            transport.close()
        pass
 
 
def create_sftp_client2(host, port, username, password, keyfilepath, keyfiletype):
    """
    create_sftp_client(host, port, username, password, keyfilepath, keyfiletype) -> SFTPClient
 
    Creates a SFTP client connected to the supplied host on the supplied port authenticating as the user with
    supplied username and supplied password or with the private key in a file with the supplied path.
    If a private key is used for authentication, the type of the keyfile needs to be specified as DSA or RSA.
    :rtype: SFTPClient object.
    """
    ssh = None
    sftp = None
    key = None
    try:
        if keyfilepath is not None:
            # Get private key used to authenticate user.
            if keyfiletype == 'DSA':
                # The private key is a DSA type key.
                key = paramiko.DSSKey.from_private_key_file(keyfilepath)
            else:
                # The private key is a RSA type key.
                key = paramiko.RSAKey.from_private_key(keyfilepath)
 
        # Connect SSH client accepting all host keys.
        ssh = paramiko.SSHClient()
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh.connect(host, port, username, password, key)
 
        # Using the SSH client, create a SFTP client.
        sftp = ssh.open_sftp()
        # Keep a reference to the SSH client in the SFTP client as to prevent the former from
        # being garbage collected and the connection from being closed.
        sftp.sshclient = ssh
 
        return sftp
    except Exception as e:
        print('An error occurred creating SFTP client: %s: %s' % (e.__class__, e))
        if sftp is not None:
            sftp.close()
        if ssh is not None:
            ssh.close()
        pass
 
host = '164.64.106.6'
port = 22
username = 'OCDsftp'
keyfile_path = None
password = 'My#1Well'
 
sftpclient = create_sftp_client(host, port, username, password, keyfile_path, 'DSA')
 
# List files in the default directory on the remote computer.
dirlist = sftpclient.listdir('/Public/OCD/')
for row in dirlist:
    print(row)
 
# Retrieve a file with the name 'remote_file.txt' on the remote computer and store it in a file named 'downloaded_file.txt'
# next to this SFT client program.
#sftpclient.get('remote_file.txt', 'downloaded_file.txt')
# Upload a file that locally has the name 'testfile.txt' to a file on the remote computer that will have the name 'remote_testfile.txt'.
#sftpclient.put('testfile.txt', 'remote_testfile.txt')
 
# We're done with the SFTPClient.
sftpclient.close()

In [ ]:
from ftplib import FTP 

ftp = FTP('164.64.106.6')

ftp.login()

ftp.cwd('/Public/OCD/')

ftp.dir()
